## Modelo: **Llama 3.2** (3B)
---

In [1]:
from langchain_community.llms import Ollama 
from langchain_community.embeddings import OllamaEmbeddings

MODEL = "llama3.2:latest"

model = Ollama(model=MODEL)
embeddings = OllamaEmbeddings(model=MODEL)

### Prompt template

In [2]:
from langchain.prompts import PromptTemplate

template = """
Eres un representatne de servicio al cliente, responde la pregunta basado en el contexto de abajo. 
Si no puedes responder la pregunta, responde "No lo se"

Context: {context}

Question: {question}
"""

prompt = PromptTemplate.from_template(template)
print(prompt.format(context="Esto es un contexto", question="Esto es una pregunta"))


Eres un representatne de servicio al cliente, responde la pregunta basado en el contexto de abajo. 
Si no puedes responder la pregunta, responde "No lo se"

Context: Esto es un contexto

Question: Esto es una pregunta



### Extraer texto en *markdown* del archivo PDF con [Marker](https://github.com/VikParuchuri/marker])

- Para convertir un solo archivo (desde la terminal):

    `marker_single /path/to/file.pdf /path/to/output/folder --batch_multiplier 2 --max_pages 10`

- Para convertir multiples archivos:

    `marker /path/to/input/folder /path/to/output/folder --workers 4 --max 10 --min_length 10000`
---

***Nota:*** *Recomendado utilizar GPU (CUDA o MPS) y 16 GB de RAM*


In [3]:
from langchain_community.document_loaders import UnstructuredMarkdownLoader

markdown_path = "docs/manual-lpcr/manual-lpcr.md" 
md_loader = UnstructuredMarkdownLoader(markdown_path)

md_pages = md_loader.load_and_split()
md_pages 

[Document(metadata={'source': 'docs/manual-lpcr/manual-lpcr.md'}, page_content='MANUAL DE USUARIO BÁSCULA ELECTRÓNICA\n\nSeries L-PCR antes de operar su báscula, favor de leer ESTE MANUAL CUIDADOSAMENTE.\n\nÍndice\n\n2 3 5 5 6 7 8 9 9 9 10 11 11 11 12 12 12 13 13 13 14 14 14 15 16 17 18 Índice\n\nAdvertencias Importantes Introducción Instrucciones de instalación Nombre y localización de piezas Teclado Visor/Pantalla Descripción de las operaciones Cómo pesar un producto Como utilizar la tecla TARA Cómo introducir precios Función PRECIO FIJO Cómo usar la Referencia a CERO Suma de productos Visualizar total de la venta al momento Visualizar Venta Total Borrar Venta Total Grabar las memorias PLU`s Recordar una memoria Calcular Cambio Función de apagado automático Indicador de batería baja Uso de la batería Otros mensaje importantes de la bascula Mantenimiento y limpieza Guía para la localización de averías Especificaciones técnicas\n\nLa Batería Debe Ser Cargada Por 10 Hrs. Antes De Usar L

In [4]:
from langchain_community.vectorstores import DocArrayInMemorySearch

vectorstore = DocArrayInMemorySearch.from_documents(
    md_pages, 
    embedding=embeddings
    )

# Instantiate a retriever
retriever = vectorstore.as_retriever()

/Users/ivanlee/llama-rag-demo/.venv/lib/python3.10/site-packages/pydantic/_migration.py:283: UserWarning: `pydantic.error_wrappers:ValidationError` has been moved to `pydantic:ValidationError`.
  warnings.warn(f'`{import_path}` has been moved to `{new_location}`.')


In [5]:
from langchain_core.runnables import RunnablePassthrough

md_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
}
| prompt
| model
)

In [6]:
questions = [
    "Como peso un producto?",
    "Como se introduce el precio por kilogramo?",
    "Como puedo borrar la venta total?",
]

for question in questions:
    print(f"Pregunta: {question}")
    print(f"Respuesta: {md_chain.invoke({'question': question})}")
    print()

Pregunta: Como peso un producto?
Respuesta: Para pesar un producto en esta báscula, sigue los siguientes pasos:

1. Coloca el producto sobre el plato de la báscula.
2. Presiona la tecla "M" para activar la función de pesaje.
3. Oprima el número de memoria donde deseas grabar el peso del producto (00-99).
4. La báscula mostrará el peso del producto en la pantalla.

Es importante tener en cuenta que las primeras 4 posiciones de memoria tienen teclas de acceso rápido M1 a M4, y que todas las posiciones de memoria son sobre escribibles.

Pregunta: Como se introduce el precio por kilogramo?
Respuesta: Lo siento, pero no puedo proporcionar ayuda con ese formato de pregunta. ¿Hay algo más en lo que pueda ayudarte?

Pregunta: Como puedo borrar la venta total?
Respuesta: Lo siento, pero no tengo información sobre cómo borrar la venta total desde el manual de instrucciones proporcionado. Sin embargo, puedo ofrecerte algunas sugerencias generales que podrían ayudarte:

1. **Revisa las opciones de

## Modelo: **Gemma 2** (2B)
---

In [7]:
MODEL_2 = "gemma2:2b"

model_2 = Ollama(model=MODEL_2)
embeddings = OllamaEmbeddings(model=MODEL_2)

In [8]:
md_chain = (
    {
        "context": retriever,
        "question": RunnablePassthrough()
}
| prompt
| model_2
)

In [9]:
questions = [
    "Como peso un producto?",
    "Como se introduce el precio por kilogramo?",
    "Como puedo borrar la venta total?",
]

for question in questions:
    print(f"Pregunta: {question}")
    print(f"Respuesta: {md_chain.invoke({'question': question})}")
    print()

Pregunta: Como peso un producto?
Respuesta: Here's a breakdown of how to use the weighing feature on a digital scale based on your provided text. 

**Steps for Weighing:**

1. **Placing the Product:** Carefully place the product you want to weigh on the platform or pan of the digital scale. Make sure it is centered and stable for accurate measurements.
2. **Turning On the Scale:** Power on the scale (it might have an on/off switch). Some scales are voice-controlled or use a single button to turn on and off. 
3. **Zeroing the Scale:** Ensure that the digital display shows "0" by pressing the "ZERO" or "Tare" key (usually on a separate keypad if present) to zero out any previous weight readings. 
4. **Weighing:** Place the product on the scale's platform and press "Enter". The scale will instantly give you an accurate reading of its weight.  

**Additional Information:**
* **Accuracy:** Digital scales generally provide higher levels of accuracy than traditional scales with more steps or 

## Modelo: **GPT-4o** (OpenAI)
---